In [1659]:
import pandas as pd
import numpy as np 
import math
from math import sqrt
import matplotlib.pyplot as plt
import collections
import missingno as msno
import warnings

import seaborn as sns
warnings.filterwarnings("ignore")

#Machine Learning
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances

# Show all columns of dataframe
pd.set_option("display.max_columns", None)

In [1660]:
# Upload all clusters
centre_back = pd.read_csv("Centre Backs 22-05-06.csv")
full_back = pd.read_csv("Full Backs 22-05-06.csv")
def_midfielder = pd.read_csv("Defensive Midfielders 22-05-06.csv")
attacker = pd.read_csv("All Attacker Clustering_22-05-06.csv")

In [1661]:
# Divide attackers into strikers and attacking midfielders 
striker = attacker[(attacker["cluster_gmm_new"] == "Target Man") | (attacker["cluster_gmm_new"] == "Finisher") | (attacker["cluster_gmm_new"] == "Hybrid Forward")]
att_midfielder = attacker[(attacker["cluster_gmm_new"] == "Attacking Creator") | (attacker["cluster_gmm_new"] == "Advanced Playmaker")]

In [1662]:
striker.replace("AM", "ST", inplace=True)

In [1663]:
# Dropping columns
#striker.drop(["% Team Touches Att Pen", "% Team Touches Att 3rd", "% Team Touches Mid 3rd"], axis=1, inplace=True)
def_midfielder.drop("Rec Rec_p100T", axis=1, inplace=True)
centre_back.drop(["Int_p100T", "Dribb Tkl_p100T", "Clr_p100T", "Blocks_p100T", "Tkl_p100T", "Press_p100T", "Recov_p100T"], axis=1, inplace=True)
full_back.drop(["Int_p100T", "Dribb Tkl_p100T", "Clr_p100T", "Blocks_p100T", "Tkl_p100T", "Press_p100T", "Recov_p100T"], axis=1, inplace=True)

# Percentile Creation

In [1665]:
# Create copies of existing DataFrames
striker_per = striker.copy()
centre_back_per = centre_back.copy()
full_back_per = full_back.copy()
def_midfielder_per = def_midfielder.copy()
att_midfielder_per = att_midfielder.copy()

In [1667]:
# Drop unnecessary columns in the middle
drops = ["Color", "Main Pos", "Poss"]
dfff = [striker_per, centre_back_per, full_back_per, def_midfielder_per, att_midfielder_per]

for df in dfff: 
    for metric in drops:
        df.drop(metric, axis=1, inplace=True)

In [1668]:
# Create Sub-DataFrames for each season 
# I could put them into dictionaries and it would look nicer but there's little value 
striker_per1 = striker_per[striker_per["Season"] == "21/22"]
striker_per2 = striker_per[striker_per["Season"] == "20/21"]
striker_per3 = striker_per[striker_per["Season"] == "19/20"]
striker_per4 = striker_per[striker_per["Season"] == "18/19"]
striker_per5 = striker_per[striker_per["Season"] == "17/18"]

centre_back_per1 = centre_back_per[centre_back_per["Season"] == "21/22"]
centre_back_per2 = centre_back_per[centre_back_per["Season"] == "20/21"]
centre_back_per3 = centre_back_per[centre_back_per["Season"] == "19/20"]
centre_back_per4 = centre_back_per[centre_back_per["Season"] == "18/19"]
centre_back_per5 = centre_back_per[centre_back_per["Season"] == "17/18"]

full_back_per1 = full_back_per[full_back_per["Season"] == "21/22"]
full_back_per2 = full_back_per[full_back_per["Season"] == "20/21"]
full_back_per3 = full_back_per[full_back_per["Season"] == "19/20"]
full_back_per4 = full_back_per[full_back_per["Season"] == "18/19"]
full_back_per5 = full_back_per[full_back_per["Season"] == "17/18"]

def_midfielder_per1 = def_midfielder_per[def_midfielder_per["Season"] == "21/22"]
def_midfielder_per2 = def_midfielder_per[def_midfielder_per["Season"] == "20/21"]
def_midfielder_per3 = def_midfielder_per[def_midfielder_per["Season"] == "19/20"]
def_midfielder_per4 = def_midfielder_per[def_midfielder_per["Season"] == "18/19"]
def_midfielder_per5 = def_midfielder_per[def_midfielder_per["Season"] == "17/18"]

att_midfielder_per1 = att_midfielder_per[att_midfielder_per["Season"] == "21/22"]
att_midfielder_per2 = att_midfielder_per[att_midfielder_per["Season"] == "20/21"]
att_midfielder_per3 = att_midfielder_per[att_midfielder_per["Season"] == "19/20"]
att_midfielder_per4 = att_midfielder_per[att_midfielder_per["Season"] == "18/19"]
att_midfielder_per5 = att_midfielder_per[att_midfielder_per["Season"] == "17/18"]

In [1669]:
# Create percentiles per position per season
strikers = [striker_per1, striker_per2, striker_per3, striker_per4, striker_per5, 
            centre_back_per1, centre_back_per2, centre_back_per3, centre_back_per4, centre_back_per5,
           full_back_per1, full_back_per2, full_back_per3, full_back_per4, full_back_per5,
           def_midfielder_per1, def_midfielder_per2, def_midfielder_per3, def_midfielder_per4, def_midfielder_per5,
           att_midfielder_per1, att_midfielder_per2, att_midfielder_per3, att_midfielder_per4, att_midfielder_per5]

for forward in strikers:
    for i in forward.iloc[:,10:-6]:
        forward[i] = ((forward[i].rank(pct=True))*100).round(2)

In [1670]:
# Concatenate DataFrames per position
striker_per = pd.concat([striker_per1, striker_per2, striker_per3, striker_per4, striker_per5])
centre_back_per = pd.concat([centre_back_per1, centre_back_per2, centre_back_per3, centre_back_per4, centre_back_per5])
full_back_per = pd.concat([full_back_per1, full_back_per2, full_back_per3, full_back_per4, full_back_per5])
def_midfielder_per = pd.concat([def_midfielder_per1, def_midfielder_per2, def_midfielder_per3,
                               def_midfielder_per4, def_midfielder_per5])
att_midfielder_per = pd.concat([att_midfielder_per1, att_midfielder_per2, att_midfielder_per3,
                               att_midfielder_per4, att_midfielder_per5])

# Create a Master DataFrame
df_per = pd.concat([striker_per, centre_back_per, full_back_per, def_midfielder_per, att_midfielder_per])


# Calculate Similarity

In [1673]:
# Concatenating the dataframes
df = pd.concat([centre_back, full_back, def_midfielder, att_midfielder, striker], axis=0)
df.head(1)

,ID,Player,Season,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,Sh,G/Sh,Dist,xG,npxG,npxG/Sh,G-xG,np:G-xG,Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Dribb Succ,Dribb Att,Dribb Succ%,Dribb #Pl,Carries,Carries TotDist,Carries PrgDist,Carries Prog,Carries 1/3,Carries PA,Mis,Dis,Rec Targ,Rec Rec,Rec Rec%,Rec Prog,Pass Att,Pass Live,Pass Dead,TB,Press Pass,Pass Sw,Crs,Corner,Ground Pass,Low Pass,High Pass,Left Pass,Right Pass,Header,Pass Cmp,Pass Cmp%,Pass TotDist,Pass PrgDist,Short Pass Cmp,Short Pass Att,Short Pass Cmp%,Med Pass Cmp,Med Pass Att,Med Pass Cmp%,Long Pass Cmp,Long Pass Att,Long Pass Cmp%,Ast,xA,A-xA,KP,Pass 1/3,Pass PA,Crs PA,Pass Prog,Tkl,TklW,Tkl Def 3rd,Tkl Mid 3rd,Tkl Att 3rd,Dribb Tkl,Dribb Tkl Att,Dribb Tkl%,Dribb Past,Press,Press Succ,Press Succ %,Press Def 3rd,Press Mid 3rd,Press Att 3rd,Blocks,Sh block,Pass block,Int,Tkl+Int,Clr,Err,Fls,Fld,Recov,AerWon,AerLost,AerWon%,SCA,SCA90,SCA PassLive,SCA PassDead,SCA Drib,Main Pos,Poss,Team Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Color,TklW %,2nd KP,Turnover,Two Footed,Sh/Touch,Crosses Succ %,Aer,1/3 Entry,PA,Touches Def Pen %,Touches Def 3rd %,Touches Mid 3rd %,Touches Att 3rd %,Touches Att Pen %,Tkl Def 3rd %,Tkl Mid 3rd %,Tkl Att 3rd %,Press Def 3rd %,Press Mid 3rd %,Press Att 3rd %,Carries_p100T,Dribb Succ_p100T,Dribb Att_p100T,Crs_p100T,KP_p100T,Pass Prog_p100T,Crs PA_p100T,Pass 1/3_p100T,xA_p100T,npxG_p100T,Carries PrgDist_p100T,Carries Prog_p100T,Carries 1/3_p100T,Carries PA_p100T,Pass PrgDist_p100T,Pass PA_p100T,Fls_p100T,Fld_p100T,Rec Prog_p100T,Pass Sw_p100T,TB_p100T,1/3 Entry_p100T,Turnover_p100T,PA_p100T,Pass Att_p100T,Aer_p100T,SCA90_p100T,Press Pass %,Pass Sw %,Short Pass Att %,Med Pass Att %,Long Pass Att %,Pass Dead %,Carries PrgDist %,Pass PrgDist %,Avg Prog Carry,Avg Prog Pass,Avg Pass Length,% Team Touches,% Team Touches_Def Pen,% Team Touches_Def 3rd,% Team Touches_Mid 3rd,% Team Touches_Att 3rd,% Team Touches_Att Pen,cluster_gmm,cluster_prob,umap1,umap2,cluster_gmm_new,cluster_prob_new,Rec Rec_p100T,Rec Targ_p100T
0,Yunis Abdelhamid (Reims 21/22),Yunis Abdelhamid,21/22,MAR,DF,Reims,Ligue 1,34,1987.0,29.1,0.07,0.62,0.11,18.6,0.04,0.04,0.07,0.03,0.03,57.1,7.7,32.1,26.2,2.85,0.89,54.1,0.48,0.65,73.7,0.48,35.7,202.3,111.8,2.89,0.72,0.0,0.48,0.34,36.9,35.8,97.1,0.34,46.9,43.6,3.26,0.0,5.77,1.48,0.03,0.0,35.2,3.88,7.84,31.5,12.1,1.44,38.5,82.1,823.2,280.4,10.4,11.5,89.9,22.3,25.1,89.2,5.46,8.93,61.2,0.0,0.01,-0.01,0.21,2.41,0.17,0.0,2.65,1.44,1.04,1.01,0.4,0.03,0.32,0.69,46.38,0.45,13.5,4.71,34.8,7.66,5.05,0.82,1.53,0.6,0.92,2.62,4.06,2.62,0.0,1.37,0.79,5.93,2.1,1.37,60.4,0.62,0.62,0.38,0.0,0.1,DF,41.9,531.7,69.4,210.8,240.2,116.4,17.6,Red,72.22,0.17,0.65,2.603306,0.010858,0.0,3.47,3.13,0.17,13.49,56.22,45.88,4.99,1.56,70.14,27.78,2.08,56.74,37.41,6.07,62.52,0.84,1.14,0.05,0.37,4.64,0.0,4.22,0.02,0.07,195.8,5.06,1.26,0.0,491.07,0.3,2.4,1.38,0.6,2.59,0.0,5.48,1.14,0.3,82.14,6.08,1.09,12.3,3.16,24.52,53.52,19.04,6.95,55.26,34.06,38.685121,105.811321,21.381818,0.107391,0.110951,0.152277,0.109076,0.024485,0.050568,CB,1.0,-4.154808,4.011469,Ball-playing Defender,0.99,NaN,NaN


## Full Backs

In [1675]:
# Create a list of metrics 
full_back = ["Touches Def Pen %", 
              "Touches Def 3rd %", 
              "Touches Mid 3rd %", 
              "Touches Att 3rd %",
              "Touches Att Pen %",
              "Carries PrgDist_p100T", 
              "Carries Prog_p100T", 
              "Carries 1/3_p100T", 
              "Carries PA_p100T",
              "Pass PrgDist_p100T",
              "Pass Prog_p100T",
              "Pass 1/3_p100T",
              "Avg Pass Length",
              "Pass PA_p100T",
              "Dribb Succ_p100T",
              "Crs_p100T", 
              "Fls_p100T",
              "AerWon", 
              "Rec Prog_p100T",    
              "xA_p100T",
              #Defensive
              "Int", 
              "Dribb Tkl",
              "Clr",
              "Blocks",
              "TklW %",
              "Recov",
              "Press Succ",
              "Press Def 3rd %", 
              "Press Mid 3rd %", 
              "Press Att 3rd %"
             ]

In [1676]:
# Create a dataframe of numerical values by selecting all numerical columns
df_full_back = df[full_back]

# Replace NULL values 
df_full_back.fillna(0, inplace=True)

In [1677]:
# Standardization
x1 = df_full_back
scaler = StandardScaler()

# Fit and then transform the data 
x_scaled1 = scaler.fit_transform(x1)
x_normed1 = pd.DataFrame(x_scaled1)

In [1678]:
# Initiate PCA
pca = PCA()

# Fit the model with x
pca.fit(x_normed1)
np.cumsum(pca.explained_variance_ratio_)

array([0.45065799, 0.58765728, 0.69071407, 0.73961583, 0.77873875,
       0.80730741, 0.83250741, 0.85351826, 0.8719348 , 0.88787647,
       0.90323603, 0.91612378, 0.92764762, 0.93727077, 0.94631625,
       0.95422299, 0.96161058, 0.96788093, 0.97320652, 0.97745802,
       0.98156799, 0.98534335, 0.98843862, 0.9913212 , 0.99408894,
       0.99640292, 0.99786084, 0.99917364, 0.99997344, 0.99999831,
       1.        ])

In [1679]:
# We will use 7 eigenvectors, (i.e. number of dimension) and project the data onto the principal components now
pca = PCA(n_components=7)

# Save components to dataframe
embedding1 = pd.DataFrame(pca.fit_transform(x_normed1))

In [1680]:
# Concat the old dataset with the PCA categories and the clusters 
df = pd.concat([df.reset_index(drop=True), embedding1], axis=1)
df.columns.values[-7: ] = ["PCA1", "PCA2", "PCA3", "PCA4", "PCA5", "PCA6", "PCA7"]
df.head(3)

,ID,Player,Season,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,Sh,G/Sh,Dist,xG,npxG,npxG/Sh,G-xG,np:G-xG,Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Dribb Succ,Dribb Att,Dribb Succ%,Dribb #Pl,Carries,Carries TotDist,Carries PrgDist,Carries Prog,Carries 1/3,Carries PA,Mis,Dis,Rec Targ,Rec Rec,Rec Rec%,Rec Prog,Pass Att,Pass Live,Pass Dead,TB,Press Pass,Pass Sw,Crs,Corner,Ground Pass,Low Pass,High Pass,Left Pass,Right Pass,Header,Pass Cmp,Pass Cmp%,Pass TotDist,Pass PrgDist,Short Pass Cmp,Short Pass Att,Short Pass Cmp%,Med Pass Cmp,Med Pass Att,Med Pass Cmp%,Long Pass Cmp,Long Pass Att,Long Pass Cmp%,Ast,xA,A-xA,KP,Pass 1/3,Pass PA,Crs PA,Pass Prog,Tkl,TklW,Tkl Def 3rd,Tkl Mid 3rd,Tkl Att 3rd,Dribb Tkl,Dribb Tkl Att,Dribb Tkl%,Dribb Past,Press,Press Succ,Press Succ %,Press Def 3rd,Press Mid 3rd,Press Att 3rd,Blocks,Sh block,Pass block,Int,Tkl+Int,Clr,Err,Fls,Fld,Recov,AerWon,AerLost,AerWon%,SCA,SCA90,SCA PassLive,SCA PassDead,SCA Drib,Main Pos,Poss,Team Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Color,TklW %,2nd KP,Turnover,Two Footed,Sh/Touch,Crosses Succ %,Aer,1/3 Entry,PA,Touches Def Pen %,Touches Def 3rd %,Touches Mid 3rd %,Touches Att 3rd %,Touches Att Pen %,Tkl Def 3rd %,Tkl Mid 3rd %,Tkl Att 3rd %,Press Def 3rd %,Press Mid 3rd %,Press Att 3rd %,Carries_p100T,Dribb Succ_p100T,Dribb Att_p100T,Crs_p100T,KP_p100T,Pass Prog_p100T,Crs PA_p100T,Pass 1/3_p100T,xA_p100T,npxG_p100T,Carries PrgDist_p100T,Carries Prog_p100T,Carries 1/3_p100T,Carries PA_p100T,Pass PrgDist_p100T,Pass PA_p100T,Fls_p100T,Fld_p100T,Rec Prog_p100T,Pass Sw_p100T,TB_p100T,1/3 Entry_p100T,Turnover_p100T,PA_p100T,Pass Att_p100T,Aer_p100T,SCA90_p100T,Press Pass %,Pass Sw %,Short Pass Att %,Med Pass Att %,Long Pass Att %,Pass Dead %,Carries PrgDist %,Pass PrgDist %,Avg Prog Carry,Avg Prog Pass,Avg Pass Length,% Team Touches,% Team Touches_Def Pen,% Team Touches_Def 3rd,% Team Touches_Mid 3rd,% Team Touches_Att 3rd,% Team Touches_Att Pen,cluster_gmm,cluster_prob,umap1,umap2,cluster_gmm_new,cluster_prob_new,Rec Rec_p100T,Rec Targ_p100T,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7
0,Yunis Abdelhamid (Reims 21/22),Yunis Abdelhamid,21/22,MAR,DF,Reims,Ligue 1,34,1987.0,29.1,0.07,0.62,0.11,18.6,0.04,0.04,0.07,0.03,0.03,57.1,7.70,32.1,26.2,2.85,0.89,54.1,0.48,0.65,73.7,0.48,35.7,202.3,111.8,2.89,0.72,0.0,0.48,0.34,36.9,35.8,97.1,0.34,46.9,43.6,3.26,0.00,5.77,1.48,0.03,0.0,35.2,3.88,7.84,31.5,12.10,1.44,38.5,82.1,823.2,280.4,10.4,11.5,89.9,22.3,25.1,89.2,5.46,8.93,61.2,0.00,0.01,-0.01,0.21,2.41,0.17,0.0,2.65,1.44,1.04,1.01,0.40,0.03,0.32,0.69,46.38,0.45,13.50,4.71,34.8,7.66,5.05,0.82,1.53,0.60,0.92,2.62,4.06,2.62,0.00,1.37,0.79,5.93,2.10,1.37,60.4,0.62,0.62,0.38,0.00,0.1,DF,41.9,531.7,69.4,210.8,240.2,116.4,17.6,Red,72.22,0.17,0.65,2.603306,0.010858,0.0,3.47,3.13,0.17,13.49,56.22,45.88,4.99,1.56,70.14,27.78,2.08,56.74,37.41,6.07,62.52,0.84,1.14,0.05,0.37,4.64,0.0,4.22,0.02,0.07,195.80,5.06,1.26,0.0,491.07,0.30,2.40,1.38,0.60,2.59,0.00,5.48,1.14,0.30,82.14,6.08,1.09,12.30,3.16,24.52,53.52,19.04,6.95,55.26,34.06,38.685121,105.811321,21.381818,0.107391,0.110951,0.152277,0.109076,0.024485,0.050568,CB,1.0,-4.154808,4.011469,Ball-playing Defender,0.99,NaN,NaN,-4.022946,-1.171251,0.307018,-0.408774,1.231675,-0.832419,-0.451108
1,Francesco Acerbi (Lazio 21/22),Francesco Acerbi,21/22,ITA,DF,Lazio,Serie A,34,1988.0,24.2,0.12,0.50,0.25,9.2,0.07,0.07,0.13,0.06,0.06,79.5,12.40,42.2,40.0,2.40,0.74,77.2,0.12,0.17,75.0,0.12,51.3,276.0,156.8,3.26,0.70,0.0,0.33,0.12,57.7,56.8,98.4,0.37,70.0,67.6,2.44,0.08,5.99,2.52,0.04,0.0,51.2,6.03,12.80,53.0,12.10,2.81,63.3,90.4,1331.2,417.8,19.7,21.1,93.3,32.6,34.6,94.3,10.10,13.00,77.8,0.00,0.02,-0.02,0.25,4.38,0.21,0.0,3.14,0.87,0.41,0.64,0.23,0.00,0.23,0.68,33.82,0.41,5.99,1.65,27.6,3.18,2.36,0.45,2.08,1.32,0.77,1.95,2.81,4.76,0.04,0.66,0.74,11.62,2.85,1.69,62.7,0.66,0.66,0.50,0.04,0.0,DF,54.8,707.7,74.8,231.1,355.8,163.7,23.3,Red,47.13,0.29,0.38,4.380165,0.006289,0.0,4.54,5.08,0.21,15.60,53.08,50.31,3.

In [1681]:
def calculate_similarity(player, squad, season):
    df['Season'].apply(lambda x: str(x))
    dist_pca1 = float(df["PCA1"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca2 = float(df["PCA2"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca3 = float(df["PCA3"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca4 = float(df["PCA4"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca5 = float(df["PCA5"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca6 = float(df["PCA6"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca7 = float(df["PCA7"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    #dist_pca8 = float(df["PCA8"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    df["Dist"] = np.sqrt( (dist_pca1 - df['PCA1'])**2 + (dist_pca2 - df['PCA2'])**2
                                 +(dist_pca3 - df['PCA3'])**2 +(dist_pca4 - df['PCA4'])**2
                        +(dist_pca5 - df['PCA5'])**2 + (dist_pca6 - df['PCA6'])**2
                        + (dist_pca7 - df['PCA7'])**2).round(2)
    df["Similarity"] = 1 - df["Dist"]/np.max(df["Dist"])
    return df[["ID", "Dist", "Similarity", "cluster_gmm_new"]].sort_values(by="Similarity", ascending=False).reset_index(drop=True).head(20)

## Centre Backs

In [1683]:
# Create a list of metrics 
centre_back = ["Carries PrgDist_p100T", 
              "Carries Prog_p100T", 
              "Carries 1/3_p100T", 
              "Pass PrgDist_p100T",
              "Pass Prog_p100T",
              "Pass Att_p100T",
              "Pass 1/3_p100T",
              "Dribb Succ_p100T",
              "Pass Sw_p100T", 
              "Avg Pass Length",
              "Pass Cmp%",
              "Fls_p100T",
              "AerWon", 
              "Int", 
              "Dribb Tkl",
              "Clr",
              "Blocks",
              "TklW %",
              "Recov",
              "Tkl Def 3rd %", 
              "Tkl Mid 3rd %", 
              "Tkl Att 3rd %",
              "Press Succ",
              "Press Def 3rd %", 
              "Press Mid 3rd %", 
              "Press Att 3rd %"
             ]

In [1684]:
# Create a dataframe of numerical values by selecting all numerical columns
df_centre_back = df[centre_back]

# Replace NULL values 
df_centre_back.fillna(0, inplace=True)

In [1685]:
# Standardization
x2 = df_centre_back
scaler = StandardScaler()

# Fit and then transform the data 
x_scaled2 = scaler.fit_transform(x2)
x_normed2 = pd.DataFrame(x_scaled2)

In [1686]:
# Initiate PCA
pca = PCA()

# Fit the model with x
pca.fit(x_normed2)
np.cumsum(pca.explained_variance_ratio_)

array([0.36096781, 0.50758435, 0.60999919, 0.6754399 , 0.71954093,
       0.7591104 , 0.79193407, 0.82104592, 0.84775492, 0.87127485,
       0.89193038, 0.90774136, 0.9232153 , 0.93640308, 0.94805474,
       0.95829256, 0.96806035, 0.97513654, 0.98203218, 0.98752144,
       0.99172882, 0.99535564, 0.99786685, 0.99980586, 0.99999798,
       1.        ])

In [1687]:
# We will use 7 eigenvectors, (i.e. number of dimension) and project the data onto the principal components now
pca = PCA(n_components=8)

# Save components to dataframe
embedding2 = pd.DataFrame(pca.fit_transform(x_normed2))

In [1688]:
# Concat the old dataset with the PCA categories and the clusters 
df = pd.concat([df.reset_index(drop=True), embedding2], axis=1)
df.columns.values[-8: ] = ["PCA1A", "PCA2A", "PCA3A", "PCA4A", "PCA5A", "PCA6A", "PCA7A", "PCA8A"]
df.head(3)

,ID,Player,Season,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,Sh,G/Sh,Dist,xG,npxG,npxG/Sh,G-xG,np:G-xG,Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Dribb Succ,Dribb Att,Dribb Succ%,Dribb #Pl,Carries,Carries TotDist,Carries PrgDist,Carries Prog,Carries 1/3,Carries PA,Mis,Dis,Rec Targ,Rec Rec,Rec Rec%,Rec Prog,Pass Att,Pass Live,Pass Dead,TB,Press Pass,Pass Sw,Crs,Corner,Ground Pass,Low Pass,High Pass,Left Pass,Right Pass,Header,Pass Cmp,Pass Cmp%,Pass TotDist,Pass PrgDist,Short Pass Cmp,Short Pass Att,Short Pass Cmp%,Med Pass Cmp,Med Pass Att,Med Pass Cmp%,Long Pass Cmp,Long Pass Att,Long Pass Cmp%,Ast,xA,A-xA,KP,Pass 1/3,Pass PA,Crs PA,Pass Prog,Tkl,TklW,Tkl Def 3rd,Tkl Mid 3rd,Tkl Att 3rd,Dribb Tkl,Dribb Tkl Att,Dribb Tkl%,Dribb Past,Press,Press Succ,Press Succ %,Press Def 3rd,Press Mid 3rd,Press Att 3rd,Blocks,Sh block,Pass block,Int,Tkl+Int,Clr,Err,Fls,Fld,Recov,AerWon,AerLost,AerWon%,SCA,SCA90,SCA PassLive,SCA PassDead,SCA Drib,Main Pos,Poss,Team Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Color,TklW %,2nd KP,Turnover,Two Footed,Sh/Touch,Crosses Succ %,Aer,1/3 Entry,PA,Touches Def Pen %,Touches Def 3rd %,Touches Mid 3rd %,Touches Att 3rd %,Touches Att Pen %,Tkl Def 3rd %,Tkl Mid 3rd %,Tkl Att 3rd %,Press Def 3rd %,Press Mid 3rd %,Press Att 3rd %,Carries_p100T,Dribb Succ_p100T,Dribb Att_p100T,Crs_p100T,KP_p100T,Pass Prog_p100T,Crs PA_p100T,Pass 1/3_p100T,xA_p100T,npxG_p100T,Carries PrgDist_p100T,Carries Prog_p100T,Carries 1/3_p100T,Carries PA_p100T,Pass PrgDist_p100T,Pass PA_p100T,Fls_p100T,Fld_p100T,Rec Prog_p100T,Pass Sw_p100T,TB_p100T,1/3 Entry_p100T,Turnover_p100T,PA_p100T,Pass Att_p100T,Aer_p100T,SCA90_p100T,Press Pass %,Pass Sw %,Short Pass Att %,Med Pass Att %,Long Pass Att %,Pass Dead %,Carries PrgDist %,Pass PrgDist %,Avg Prog Carry,Avg Prog Pass,Avg Pass Length,% Team Touches,% Team Touches_Def Pen,% Team Touches_Def 3rd,% Team Touches_Mid 3rd,% Team Touches_Att 3rd,% Team Touches_Att Pen,cluster_gmm,cluster_prob,umap1,umap2,cluster_gmm_new,cluster_prob_new,Rec Rec_p100T,Rec Targ_p100T,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,Similarity,PCA1A,PCA2A,PCA3A,PCA4A,PCA5A,PCA6A,PCA7A,PCA8A
0,Yunis Abdelhamid (Reims 21/22),Yunis Abdelhamid,21/22,MAR,DF,Reims,Ligue 1,34,1987.0,29.1,0.07,0.62,0.11,6.04,0.04,0.04,0.07,0.03,0.03,57.1,7.70,32.1,26.2,2.85,0.89,54.1,0.48,0.65,73.7,0.48,35.7,202.3,111.8,2.89,0.72,0.0,0.48,0.34,36.9,35.8,97.1,0.34,46.9,43.6,3.26,0.00,5.77,1.48,0.03,0.0,35.2,3.88,7.84,31.5,12.10,1.44,38.5,82.1,823.2,280.4,10.4,11.5,89.9,22.3,25.1,89.2,5.46,8.93,61.2,0.00,0.01,-0.01,0.21,2.41,0.17,0.0,2.65,1.44,1.04,1.01,0.40,0.03,0.32,0.69,46.38,0.45,13.50,4.71,34.8,7.66,5.05,0.82,1.53,0.60,0.92,2.62,4.06,2.62,0.00,1.37,0.79,5.93,2.10,1.37,60.4,0.62,0.62,0.38,0.00,0.1,DF,41.9,531.7,69.4,210.8,240.2,116.4,17.6,Red,72.22,0.17,0.65,2.603306,0.010858,0.0,3.47,3.13,0.17,13.49,56.22,45.88,4.99,1.56,70.14,27.78,2.08,56.74,37.41,6.07,62.52,0.84,1.14,0.05,0.37,4.64,0.0,4.22,0.02,0.07,195.80,5.06,1.26,0.0,491.07,0.30,2.40,1.38,0.60,2.59,0.00,5.48,1.14,0.30,82.14,6.08,1.09,12.30,3.16,24.52,53.52,19.04,6.95,55.26,34.06,38.685121,105.811321,21.381818,0.107391,0.110951,0.152277,0.109076,0.024485,0.050568,CB,1.0,-4.154808,4.011469,Ball-playing Defender,0.99,NaN,NaN,-4.022946,-1.171251,0.307018,-0.408774,1.231675,-0.832419,-0.451108,0.522907,-2.760948,0.788773,0.397843,0.062560,-0.415753,-1.307887,-0.437977,-0.163045
1,Francesco Acerbi (Lazio 21/22),Francesco Acerbi,21/22,ITA,DF,Lazio,Serie A,34,1988.0,24.2,0.12,0.50,0.25,6.03,0.07,0.07,0.13,0.06,0.06,79.5,12.40,42.2,40.0,2.40,0.74,77.2,0.12,0.17,75.0,0.12,51.3,276.0,156.8,3.26,0.70,0.0,0.33,0.12,57.7,56.8,98.4,0.37,70.0,67.6,2.44,0.08,5.99,2.52,0.04,0.0,51.2,6.03,12.80,53.0,12.10,2.81,63.3,90.4,1331.2,417.8,19.7,21.1,93.3,32.6,34.6,94.3,10.10,13.00,77.8,0.00,0.02,-0.02,0.25,4.38,0.21,0.0,3.14,0.87,0.41,0.64,0.23,0.00,0.23,0.68,33.82,0.41,5.99,1.65,27.6,3.18,2.36,0.45,2.08,1.32,0.77,1.95,2.81,4.76,0.04,0.66,0.74,11.62,2.85,1.69,62

In [1689]:
def calculate_similarity1(player, squad, season):
    df['Season'].apply(lambda x: str(x))
    dist_pca1A = float(df["PCA1A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca2A = float(df["PCA2A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca3A = float(df["PCA3A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca4A = float(df["PCA4A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca5A = float(df["PCA5A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca6A = float(df["PCA6A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca7A = float(df["PCA7A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca8A = float(df["PCA8A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    df["DistA"] = np.sqrt( (dist_pca1A - df['PCA1A'])**2 + (dist_pca2A - df['PCA2A'])**2
                                 +(dist_pca3A - df['PCA3A'])**2 +(dist_pca4A - df['PCA4A'])**2
                        +(dist_pca5A - df['PCA5A'])**2 + (dist_pca6A - df['PCA6A'])**2
                        + (dist_pca7A - df['PCA7A'])**2 + (dist_pca8A - df['PCA8A'])**2).round(2)
    df["SimilarityA"] = 1 - df["DistA"]/np.max(df["DistA"])
    return df[["ID", "DistA", "SimilarityA", "cluster_gmm_new"]].sort_values(by="SimilarityA", ascending=False).reset_index(drop=True).head(50)

## Cluster Defensive Midfielder

In [1691]:
# Create a list of metrics 
defmf = [     "% Team Touches",
              "Touches Def 3rd %", 
              "Touches Mid 3rd %", 
              "Touches Att 3rd %",
              "Touches Att Pen %",
              "Carries PrgDist_p100T", 
              "Carries Prog_p100T", 
              "Carries 1/3_p100T", 
              "Carries PA_p100T",
              "Pass PrgDist_p100T",
              "Pass 1/3_p100T",
              "Pass Prog_p100T",
              "Pass PA_p100T",
              "Pass Att_p100T",
              "Dribb Succ_p100T",
              "Crs_p100T", 
              "Pass Sw_p100T",
              "Avg Pass Length",
              "Sh/Touch", 
              "Pass Cmp%",
              "Fls_p100T",
              "AerWon", 
              "Turnover_p100T",
              "Rec Prog_p100T",
              "npxG_p100T",
              "xA_p100T",
              #Defensive
              "Int", 
              "Dribb Tkl",
              "Blocks",
              "Recov",
              "Press Succ",
              "Press Def 3rd %", 
              "Press Mid 3rd %", 
              "Press Att 3rd %"
             ]

In [1692]:
# Create a dataframe of numerical values by selecting all numerical columns
df_def_midfielder = df[defmf]

# Replace NULL values 
df_def_midfielder.fillna(0, inplace=True)

In [1693]:
# Standardization
x3 = df_def_midfielder
scaler = StandardScaler()

# Fit and then transform the data 
x_scaled3 = scaler.fit_transform(x3)
x_normed3 = pd.DataFrame(x_scaled3)

In [1694]:
# Initiate PCA
pca = PCA()

# Fit the model with x
pca.fit(x_normed3)
np.cumsum(pca.explained_variance_ratio_)

array([0.450083  , 0.58796264, 0.67967051, 0.73088385, 0.77484606,
       0.80617862, 0.83067617, 0.85228913, 0.86967376, 0.88495197,
       0.8977186 , 0.90911418, 0.92002889, 0.92932182, 0.93761561,
       0.94532241, 0.95259772, 0.95929785, 0.9654983 , 0.97029505,
       0.97470297, 0.97895873, 0.98256755, 0.98592352, 0.98891769,
       0.991463  , 0.9936777 , 0.99574023, 0.99713491, 0.99832702,
       0.99923511, 0.99997605, 0.99999846, 1.        ])

In [1695]:
# We will use 7 eigenvectors, (i.e. number of dimension) and project the data onto the principal components now
pca = PCA(n_components=7)

# Save components to dataframe
embedding3 = pd.DataFrame(pca.fit_transform(x_normed3))

In [1696]:
# Concat the old dataset with the PCA categories and the clusters 
df = pd.concat([df.reset_index(drop=True), embedding3], axis=1)
df.columns.values[-7: ] = ["PCA1B", "PCA2B", "PCA3B", "PCA4B", "PCA5B", "PCA6B", "PCA7B"]
df.head(3)

,ID,Player,Season,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,Sh,G/Sh,Dist,xG,npxG,npxG/Sh,G-xG,np:G-xG,Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Dribb Succ,Dribb Att,Dribb Succ%,Dribb #Pl,Carries,Carries TotDist,Carries PrgDist,Carries Prog,Carries 1/3,Carries PA,Mis,Dis,Rec Targ,Rec Rec,Rec Rec%,Rec Prog,Pass Att,Pass Live,Pass Dead,TB,Press Pass,Pass Sw,Crs,Corner,Ground Pass,Low Pass,High Pass,Left Pass,Right Pass,Header,Pass Cmp,Pass Cmp%,Pass TotDist,Pass PrgDist,Short Pass Cmp,Short Pass Att,Short Pass Cmp%,Med Pass Cmp,Med Pass Att,Med Pass Cmp%,Long Pass Cmp,Long Pass Att,Long Pass Cmp%,Ast,xA,A-xA,KP,Pass 1/3,Pass PA,Crs PA,Pass Prog,Tkl,TklW,Tkl Def 3rd,Tkl Mid 3rd,Tkl Att 3rd,Dribb Tkl,Dribb Tkl Att,Dribb Tkl%,Dribb Past,Press,Press Succ,Press Succ %,Press Def 3rd,Press Mid 3rd,Press Att 3rd,Blocks,Sh block,Pass block,Int,Tkl+Int,Clr,Err,Fls,Fld,Recov,AerWon,AerLost,AerWon%,SCA,SCA90,SCA PassLive,SCA PassDead,SCA Drib,Main Pos,Poss,Team Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Color,TklW %,2nd KP,Turnover,Two Footed,Sh/Touch,Crosses Succ %,Aer,1/3 Entry,PA,Touches Def Pen %,Touches Def 3rd %,Touches Mid 3rd %,Touches Att 3rd %,Touches Att Pen %,Tkl Def 3rd %,Tkl Mid 3rd %,Tkl Att 3rd %,Press Def 3rd %,Press Mid 3rd %,Press Att 3rd %,Carries_p100T,Dribb Succ_p100T,Dribb Att_p100T,Crs_p100T,KP_p100T,Pass Prog_p100T,Crs PA_p100T,Pass 1/3_p100T,xA_p100T,npxG_p100T,Carries PrgDist_p100T,Carries Prog_p100T,Carries 1/3_p100T,Carries PA_p100T,Pass PrgDist_p100T,Pass PA_p100T,Fls_p100T,Fld_p100T,Rec Prog_p100T,Pass Sw_p100T,TB_p100T,1/3 Entry_p100T,Turnover_p100T,PA_p100T,Pass Att_p100T,Aer_p100T,SCA90_p100T,Press Pass %,Pass Sw %,Short Pass Att %,Med Pass Att %,Long Pass Att %,Pass Dead %,Carries PrgDist %,Pass PrgDist %,Avg Prog Carry,Avg Prog Pass,Avg Pass Length,% Team Touches,% Team Touches_Def Pen,% Team Touches_Def 3rd,% Team Touches_Mid 3rd,% Team Touches_Att 3rd,% Team Touches_Att Pen,cluster_gmm,cluster_prob,umap1,umap2,cluster_gmm_new,cluster_prob_new,Rec Rec_p100T,Rec Targ_p100T,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,Similarity,PCA1A,PCA2A,PCA3A,PCA4A,PCA5A,PCA6A,PCA7A,PCA8A,DistA,SimilarityA,PCA1B,PCA2B,PCA3B,PCA4B,PCA5B,PCA6B,PCA7B
0,Yunis Abdelhamid (Reims 21/22),Yunis Abdelhamid,21/22,MAR,DF,Reims,Ligue 1,34,1987.0,29.1,0.07,0.62,0.11,6.04,0.04,0.04,0.07,0.03,0.03,57.1,7.70,32.1,26.2,2.85,0.89,54.1,0.48,0.65,73.7,0.48,35.7,202.3,111.8,2.89,0.72,0.0,0.48,0.34,36.9,35.8,97.1,0.34,46.9,43.6,3.26,0.00,5.77,1.48,0.03,0.0,35.2,3.88,7.84,31.5,12.10,1.44,38.5,82.1,823.2,280.4,10.4,11.5,89.9,22.3,25.1,89.2,5.46,8.93,61.2,0.00,0.01,-0.01,0.21,2.41,0.17,0.0,2.65,1.44,1.04,1.01,0.40,0.03,0.32,0.69,46.38,0.45,13.50,4.71,34.8,7.66,5.05,0.82,1.53,0.60,0.92,2.62,4.06,2.62,0.00,1.37,0.79,5.93,2.10,1.37,60.4,0.62,0.62,0.38,0.00,0.1,DF,41.9,531.7,69.4,210.8,240.2,116.4,17.6,Red,72.22,0.17,0.65,2.603306,0.010858,0.0,3.47,3.13,0.17,13.49,56.22,45.88,4.99,1.56,70.14,27.78,2.08,56.74,37.41,6.07,62.52,0.84,1.14,0.05,0.37,4.64,0.0,4.22,0.02,0.07,195.80,5.06,1.26,0.0,491.07,0.30,2.40,1.38,0.60,2.59,0.00,5.48,1.14,0.30,82.14,6.08,1.09,12.30,3.16,24.52,53.52,19.04,6.95,55.26,34.06,38.685121,105.811321,21.381818,0.107391,0.110951,0.152277,0.109076,0.024485,0.050568,CB,1.0,-4.154808,4.011469,Ball-playing Defender,0.99,NaN,NaN,-4.022946,-1.171251,0.307018,-0.408774,1.231675,-0.832419,-0.451108,0.522907,-2.760948,0.788773,0.397843,0.062560,-0.415753,-1.307887,-0.437977,-0.163045,4.44,0.697342,-3.803934,1.534017,0.851200,0.004384,0.527582,-0.811426,0.583756
1,Francesco Acerbi (Lazio 21/22),Francesco Acerbi,21/22,ITA,DF,Lazio,Serie A,34,1988.0,24.2,0.12,0.50,0.25,6.03,0.07,0.07,0.13,0.06,0.06,79.5,12.40,42.2,40.0,2.40,0.74,77.2,0.12,0.17,75.0,0.12,51.3,276.0,156.8,3.26,0.70,0.0,0.33,0.12,57.7,56.8,98.4,0.37,70.0,67.6,2.44,0.08,5.99,2.52,0.04,0.0,51.2,6.03,12.80,53.0,12.10,2.81,63.3,90.4,1331.2,417.8,19.7,21.1,93.3,32.6,34.6,94.3,10.10,13.00,77.8,0.00,0.02,-0.02,0.25,4.38,0.21,0.0,3.14,

In [1697]:
def calculate_similarity2(player, squad, season):
    df['Season'].apply(lambda x: str(x))
    dist_pca1B = float(df["PCA1B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca2B = float(df["PCA2B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca3B = float(df["PCA3B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca4B = float(df["PCA4B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca5B = float(df["PCA5B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca6B = float(df["PCA6B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca7B = float(df["PCA7B"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    #dist_pca8B = float(df["PCA8A"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    df["DistB"] = np.sqrt( (dist_pca1B - df['PCA1B'])**2 + (dist_pca2B - df['PCA2B'])**2
                                 +(dist_pca3B - df['PCA3B'])**2 +(dist_pca4B - df['PCA4B'])**2
                        +(dist_pca5B - df['PCA5B'])**2 + (dist_pca6B - df['PCA6B'])**2
                        + (dist_pca7B - df['PCA7B'])**2).round(2) #+ (dist_pca8A - df['PCA8A'])**2).round(2)
    df["SimilarityB"] = 1 - df["DistB"]/np.max(df["DistB"])
    return df[["ID", "DistB", "SimilarityB", "cluster_gmm_new"]].sort_values(by="SimilarityB", ascending=False).reset_index(drop=True).head(20)

# New attack

In [1699]:
# Create a list of metrics 
att_midfielder = ["% Team Touches",
              "Touches Mid 3rd %", 
              "Touches Att 3rd %",
              "Touches Att Pen %",
              "Carries PrgDist_p100T", 
              "Carries Prog_p100T", 
              "Carries 1/3_p100T", 
              "Carries PA_p100T",
              "Pass PrgDist_p100T",
              "Pass 1/3_p100T",
              "Pass Att_p100T",
              "Pass PA_p100T",
              "Pass Prog_p100T",
              "Dribb Succ_p100T",
              "Crs_p100T", 
              "Pass Sw_p100T",
              "Sh/Touch", 
              "Pass Cmp%",
              "AerWon", 
              "Rec Prog_p100T",
              "npxG_p100T",
              "xA_p100T",
             ]

In [1700]:
# Create a dataframe of numerical values by selecting all numerical columns
df_att_midfielder = df[att_midfielder]

# Replace NULL values 
df_att_midfielder.fillna(0, inplace=True)

In [1701]:
# Standardization
x6 = df_att_midfielder
scaler = StandardScaler()

# Fit and then transform the data 
x_scaled6 = scaler.fit_transform(x6)
x_normed6 = pd.DataFrame(x_scaled6)

In [1702]:
# Initiate PCA
pca = PCA()

# Fit the model with x
pca.fit(x_normed6)
np.cumsum(pca.explained_variance_ratio_)

array([0.44806671, 0.64759319, 0.72625275, 0.78718675, 0.82440075,
       0.85951472, 0.89186134, 0.90851107, 0.92270771, 0.93594941,
       0.94705078, 0.9560276 , 0.96320081, 0.97006295, 0.97646526,
       0.98199318, 0.98683147, 0.99119124, 0.99466899, 0.99744292,
       0.99887435, 1.        ])

In [1703]:
# We will use 7 eigenvectors, (i.e. number of dimension) and project the data onto the principal components now
pca = PCA(n_components=6)

# Save components to dataframe
embedding6 = pd.DataFrame(pca.fit_transform(x_normed6))

In [1704]:
# Concat the old dataset with the PCA categories and the clusters 
df = pd.concat([df.reset_index(drop=True), embedding6], axis=1)
df.columns.values[-6: ] = ["PCA1E", "PCA2E", "PCA3E", "PCA4E", "PCA5E", "PCA6E"]
df.head(3)

,ID,Player,Season,Nation,Pos,Squad,Comp,Age,Born,90s,Gls,Sh,G/Sh,Dist,xG,npxG,npxG/Sh,G-xG,np:G-xG,Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Dribb Succ,Dribb Att,Dribb Succ%,Dribb #Pl,Carries,Carries TotDist,Carries PrgDist,Carries Prog,Carries 1/3,Carries PA,Mis,Dis,Rec Targ,Rec Rec,Rec Rec%,Rec Prog,Pass Att,Pass Live,Pass Dead,TB,Press Pass,Pass Sw,Crs,Corner,Ground Pass,Low Pass,High Pass,Left Pass,Right Pass,Header,Pass Cmp,Pass Cmp%,Pass TotDist,Pass PrgDist,Short Pass Cmp,Short Pass Att,Short Pass Cmp%,Med Pass Cmp,Med Pass Att,Med Pass Cmp%,Long Pass Cmp,Long Pass Att,Long Pass Cmp%,Ast,xA,A-xA,KP,Pass 1/3,Pass PA,Crs PA,Pass Prog,Tkl,TklW,Tkl Def 3rd,Tkl Mid 3rd,Tkl Att 3rd,Dribb Tkl,Dribb Tkl Att,Dribb Tkl%,Dribb Past,Press,Press Succ,Press Succ %,Press Def 3rd,Press Mid 3rd,Press Att 3rd,Blocks,Sh block,Pass block,Int,Tkl+Int,Clr,Err,Fls,Fld,Recov,AerWon,AerLost,AerWon%,SCA,SCA90,SCA PassLive,SCA PassDead,SCA Drib,Main Pos,Poss,Team Touches,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Color,TklW %,2nd KP,Turnover,Two Footed,Sh/Touch,Crosses Succ %,Aer,1/3 Entry,PA,Touches Def Pen %,Touches Def 3rd %,Touches Mid 3rd %,Touches Att 3rd %,Touches Att Pen %,Tkl Def 3rd %,Tkl Mid 3rd %,Tkl Att 3rd %,Press Def 3rd %,Press Mid 3rd %,Press Att 3rd %,Carries_p100T,Dribb Succ_p100T,Dribb Att_p100T,Crs_p100T,KP_p100T,Pass Prog_p100T,Crs PA_p100T,Pass 1/3_p100T,xA_p100T,npxG_p100T,Carries PrgDist_p100T,Carries Prog_p100T,Carries 1/3_p100T,Carries PA_p100T,Pass PrgDist_p100T,Pass PA_p100T,Fls_p100T,Fld_p100T,Rec Prog_p100T,Pass Sw_p100T,TB_p100T,1/3 Entry_p100T,Turnover_p100T,PA_p100T,Pass Att_p100T,Aer_p100T,SCA90_p100T,Press Pass %,Pass Sw %,Short Pass Att %,Med Pass Att %,Long Pass Att %,Pass Dead %,Carries PrgDist %,Pass PrgDist %,Avg Prog Carry,Avg Prog Pass,Avg Pass Length,% Team Touches,% Team Touches_Def Pen,% Team Touches_Def 3rd,% Team Touches_Mid 3rd,% Team Touches_Att 3rd,% Team Touches_Att Pen,cluster_gmm,cluster_prob,umap1,umap2,cluster_gmm_new,cluster_prob_new,Rec Rec_p100T,Rec Targ_p100T,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,Similarity,PCA1A,PCA2A,PCA3A,PCA4A,PCA5A,PCA6A,PCA7A,PCA8A,DistA,SimilarityA,PCA1B,PCA2B,PCA3B,PCA4B,PCA5B,PCA6B,PCA7B,DistB,SimilarityB,PCA1E,PCA2E,PCA3E,PCA4E,PCA5E,PCA6E
0,Yunis Abdelhamid (Reims 21/22),Yunis Abdelhamid,21/22,MAR,DF,Reims,Ligue 1,34,1987.0,29.1,0.07,0.62,0.11,6.04,0.04,0.04,0.07,0.03,0.03,57.1,7.70,32.1,26.2,2.85,0.89,54.1,0.48,0.65,73.7,0.48,35.7,202.3,111.8,2.89,0.72,0.0,0.48,0.34,36.9,35.8,97.1,0.34,46.9,43.6,3.26,0.00,5.77,1.48,0.03,0.0,35.2,3.88,7.84,31.5,12.10,1.44,38.5,82.1,823.2,280.4,10.4,11.5,89.9,22.3,25.1,89.2,5.46,8.93,61.2,0.00,0.01,-0.01,0.21,2.41,0.17,0.0,2.65,1.44,1.04,1.01,0.40,0.03,0.32,0.69,46.38,0.45,13.50,4.71,34.8,7.66,5.05,0.82,1.53,0.60,0.92,2.62,4.06,2.62,0.00,1.37,0.79,5.93,2.10,1.37,60.4,0.62,0.62,0.38,0.00,0.1,DF,41.9,531.7,69.4,210.8,240.2,116.4,17.6,Red,72.22,0.17,0.65,2.603306,0.010858,0.0,3.47,3.13,0.17,13.49,56.22,45.88,4.99,1.56,70.14,27.78,2.08,56.74,37.41,6.07,62.52,0.84,1.14,0.05,0.37,4.64,0.0,4.22,0.02,0.07,195.80,5.06,1.26,0.0,491.07,0.30,2.40,1.38,0.60,2.59,0.00,5.48,1.14,0.30,82.14,6.08,1.09,12.30,3.16,24.52,53.52,19.04,6.95,55.26,34.06,38.685121,105.811321,21.381818,0.107391,0.110951,0.152277,0.109076,0.024485,0.050568,CB,1.0,-4.154808,4.011469,Ball-playing Defender,0.99,NaN,NaN,-4.022946,-1.171251,0.307018,-0.408774,1.231675,-0.832419,-0.451108,0.522907,-2.760948,0.788773,0.397843,0.062560,-0.415753,-1.307887,-0.437977,-0.163045,4.44,0.697342,-3.803934,1.534017,0.851200,0.004384,0.527582,-0.811426,0.583756,6.47,0.520030,-2.853274,-1.069460,0.743839,0.830809,0.648689,0.425089
1,Francesco Acerbi (Lazio 21/22),Francesco Acerbi,21/22,ITA,DF,Lazio,Serie A,34,1988.0,24.2,0.12,0.50,0.25,6.03,0.07,0.07,0.13,0.06,0.06,79.5,12.40,42.2,40.0,2.40,0.74,77.2,0.12,0.17,75.0,0.12,51.3,276.0,156.8,3.26,0.70,0.0,0.33,0.12,57.7,56.8,98.4,0.37,70.0,67.6,2.44,0.08,5.99,2.52,0.04,0.0,51.2,6.03,12.80,53

In [1705]:
def calculate_similarity6(player, squad, season):
    df['Season'].apply(lambda x: str(x))
    dist_pca1E = float(df["PCA1E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca2E = float(df["PCA2E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca3E = float(df["PCA3E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca4E = float(df["PCA4E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca5E = float(df["PCA5E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    dist_pca6E = float(df["PCA6E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    #dist_pca7C = float(df["PCA7C"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    #dist_pca8E = float(df["PCA8E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    #dist_pca9E = float(df["PCA9E"][(df["ID"] == player + " (" + squad + " " + season + ")")])
    df["DistE"] = np.sqrt( (dist_pca1E - df['PCA1E'])**2 + (dist_pca2E - df['PCA2E'])**2
                                 +(dist_pca3E - df['PCA3E'])**2 +(dist_pca4E - df['PCA4E'])**2
                        +(dist_pca5E - df['PCA5E'])**2 + (dist_pca6E - df['PCA6E'])**2
                        #+ (dist_pca7C - df['PCA7C'])**2
                         ).round(2)
    #+ (dist_pca8E - df['PCA8E'])**2 + (dist_pca9E - df['PCA9E'])**2).round(2)
    df["SimilarityE"] = 1 - df["DistE"]/np.max(df["DistE"])
    return df[["ID", "DistE", "SimilarityE", "cluster_gmm_new"]].sort_values(by="SimilarityE", ascending=False).reset_index(drop=True).head(50)

In [1709]:
df.to_csv("Similar Player_22-05-11.csv", index=False)

In [1710]:
df_per.to_csv("Similar Player Percentiles_22-05-11.csv", index=False)